# Introduction

A rédiger

# Imports

In [1]:
import numpy as np
import pandas as pd 
import datetime as dt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Loading the dataset

In [2]:
def loader (excel) :
    
    df = pd.DataFrame(pd.read_excel(excel))
    
    df = df.drop(["Unnamed: 0","date_added"], axis = 1)
    
    return df

streaming_content = loader('streaming_content.xlsx')

# Customization (optional)

A rédiger

### Platforms

A remettre en forme + photo

First, in which platforms do you want to obtain recommendations ? Netflix, Disney +, Amazon or all ?

Possible answers :

Netflix, Disney +, Prime Video, Hulu, all.

In [3]:
filter_platform = str(input("Do you want recommendations based on all the platforms ? : "))

list_platform = []

if filter_platform == 'no' :
    
    ptf_number = int(input("Number of non desired platforms ? : "))
    
    for i in range(0,ptf_number) :
        
        platform_i = str(input("Non desired platform : "))
        
        list_platform.append(platform_i)

Do you want recommendations based on all the platforms ? : yes


### Rating IMDb

A remettre en forme + photo

Expliquer ce qu'est IMDb

Secondly, what is the minimum grade on IMDb the movie or the TV show must have ? (Data not available for all the dataset) ? :

0.1 to 10, float number granted (0 means that the IMDb rating is not a relevant feature).

--------------------------

Thirdly, what is the minimum of IMDb reviews the movie or the TV show must have ? :

No limitation (0 means that the IMDb rating is not a relevant feature).

In [4]:
rating = float(input("Minimum rating on IMDb ? : "))

rewiews = int(input("Minimum number of reviews ? : "))

Minimum rating on IMDb ? : 0
Minimum number of reviews ? : 0


### Country

A remettre en forme + photo

Please select a country, if you have no preference, please enter 0. 

In [5]:
country = str(input("Selected country : "))

Selected country : 0


### Release year

A remettre en forme + photo ? 

Please select a year and precise if you want the shows of this year or the shows of this year and the folowing ones (equal vs minimum). 

Enter 0 in one or both categories if year is not an important feature for you. 

In [6]:
year = int(input("Enter a release year : "))
quality = str(input("Show of the year ? Minimum year of release ? : "))

Enter a release year : 2010
Show of the year ? Minimum year of release ? : minimum


### Type

A remettre en forme + photo

Do you want recommendations for TV Show or for Movies ? (tvshow, movie, all)

In [7]:
show = str(input("TV Show, Movie or all : "))

TV Show, Movie or all : all



### Classification

A remettre en forme + photo ? 

For this algorithm, we use a simplified version of the American rating :

Not Rated, All ages, 7+, PG-13, 16+, R-Rated

In [8]:
list_filter_classification = []
filter_classification = str(input("Do you want any filter ? : "))

if filter_classification == "yes":
    
    number_filter = int(input("Number of classification filter : "))
    
    for i in range(0,number_filter): 
        
        class_i = str(input("Non-Desired classification : "))
        
        list_filter_classification.append(class_i)

Do you want any filter ? : yes
Number of classification filter : 2
Non-Desired classification : All ages
Non-Desired classification : 7+


### Category

A remettre en forme + mettre une image + lien vers liste de catégorie

Please enter how many filter in category do you want to apply (negative filter). 

Please enter the filter with the first letter in capital. 

In [9]:
filter_number = int(input("Number of category filter : "))
list_filter_category = []

for i in range(0, filter_number) :
    
    filter_i = str(input("Enter a non desired category : "))
    
    list_filter_category.append(filter_i)

Number of category filter : 2
Enter a non desired category : Young
Enter a non desired category : kid


# Content Based Recommendation Algorithm

Expliquer le principe du content based et du cosinus de similarité.

But rentrer le nom d'une série ou d'un film et rechercher le contenu le plus similaire en fonction du dataset d'entrée. 

### Data Cleaning

Biais => si categorie proposera automatiquement plus d'une plateforme (perte diversité) car catégorie propre à chaque plateforme. 

In [10]:
#features = ["title","director","cast","listed_in","description"]
features = ["title","director","cast","description"]
#features = ["title","description"]
#features = ["title","director","description"]

In [11]:
reco_streaming = streaming_content[features]

In [12]:
def fillna(df) :
    
    return df.fillna(" ")

reco_streaming = fillna(reco_streaming)

In [13]:
def clean_data(col):
        return str.lower(col.replace(" ", ""))


Tout mettre en minuscule et supprimer les espaces, uniformisation des données textuelles.

In [14]:
def cleaner(df, feat) :
    
    for feature in feat:
        df[feature] = df[feature].apply(clean_data)
        
    return df

reco_streaming = cleaner(reco_streaming, features)

### Creation of a soup 

Soup = bag of words, représentation du corpus => fusionner tous les mots pour récapituler un show et toutes les information dessus, les stop words seront supprimés. 

In [15]:
def create_soup(df):
    
    #df["soup"] = df['title']+ ' ' + df['director'] + ' ' + df['cast'] + ' ' +df['listed_in']+' '+ df['description']
    df["soup"] = df['title']+ ' ' + df['director'] + ' ' + df['cast'] + ' '+ df['description']
    #df["soup"] = df['title']+ ' ' + df['description']
    #df["soup"] = df['title']+ ' ' + df['director'] + ' '+ df['description']
    return df

reco_streaming = create_soup(reco_streaming)

### Pre-processing

In [16]:
def pre_processing (df) :

    count = CountVectorizer(stop_words='english')
    count_matrix = count.fit_transform(df['soup'])

    cos_sim = cosine_similarity(count_matrix, count_matrix)
    
    df = df.reset_index()
    indices = pd.Series(df.index, index = df['title'])
    
    return df, cos_sim, indices

reco_streaming, cosine_sim, index = pre_processing(reco_streaming)

### Similarity

Faire schéma explicatif : sortie cosine sim donne pour un index i la similitude selon les critères les colonnes sont les index j. 

Quand on rentre un titre, il est uniformisé et un résultat en sortie est fourni avec une recherche par index dans le cosine sim. 

A la fin on fait remonter les informations clés en matchant avec le dataframe de base.

In [18]:
def similarity (title, cos_sim, idx, df1) :
    
    title=title.replace(' ','').lower()
    idx = idx[title]
    
    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cos_sim[idx]))
    
    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]
    
    return sim_scores

simil = similarity(input('Enter a movie or a TV show name : '), cosine_sim, index, streaming_content)

Enter a movie or a TV show name : Django Unchained


In [19]:
def dataframe_simil(sim):
    
    df = pd.DataFrame(sim, columns = ["index","similarity"])
    
    return df

similarity = dataframe_simil(simil)

### Filtering dataset

In [20]:
def new_set(xlsx, df1) :
    
    df = loader(xlsx)
    
    df = df.reset_index()
    
    #Merging similarities
    
    df = df.merge(df1, how = 'left', left_on = 'index', right_on = 'index')
    
    return df

recommendation = new_set("streaming_content.xlsx", similarity)

In [21]:
def filtering_plateforms (df, list_ptf) :
    
    for i in range(0,len(list_ptf)) :
        
        df = df[df["origin"]!= list_ptf[i]]
        
    return df

recommendation = filtering_plateforms(recommendation, list_platform)

In [22]:
def filtering_imdb (df, rating, review) :
    
    if rating == 0 and review == 0 : 
        
        df = df
        
    elif rating != 0 and review == 0 :
        
        df = df[df["averageRating"] >= rating]
        
    elif rating == 0 and review != 0 :
        
        df = df[df["numVotes"] >= review]
        
    else :
        
        df = df[(df["averageRating"] >= rating) & (df["numVotes"] >= review)]
        
    return df

recommendation = filtering_imdb(recommendation, rating, rewiews)

In [23]:
def filtering_country(df, cnt):
    
    if cnt == "0" :
        
        df = df 
    
    else :
        
        df = df[df["country"]==cnt]
    
    return df

recommendation = filtering_country(recommendation, country)

In [24]:
def filtering_year (df, year, qual) :
        
    if (year != 0) and (qual == "equal") :
        
        df = df[df["release_year"] == year]
        
    elif (year != 0) and (qual == "minimum") :
        
        df = df[df["release_year"] >= year]
        
    else : 
        
        df = df
        
    return df

recommendation = filtering_year(recommendation, year, quality)

In [25]:
def filtering_type (df, shw):
    
    if shw == "tvshow":
        
        df = df[df["type"] == "TV Show"]
        
    elif shw == "movie":
        
        df = df[df["type"] == "Movie"]
        
    else :
        
        df = df
        
    return df

recommendation = filtering_type(recommendation, show)

In [26]:
def filtering_classification (df, list_classif) :
    
    for i in range(0,len(list_classif)) :
        
        df = df[df["rating"]!= list_classif[i]]
        
    return df

recommendation = filtering_classification(recommendation, list_filter_classification)

In [27]:
def filtering_category (df, filter_list) :
    
    for i in range(0,len(filter_list)) :
        
        df = df[~df["listed_in"].str.contains(filter_list[i])]
        
    return df

recommendation = filtering_category(recommendation, list_filter_category)

### Recommendation

In [28]:
def get_recommendations(df1) :
    
    df = df1.sort_values(by = ["similarity"], ascending = False)
    
    df = df[["title","description", "averageRating","origin","type"]].head(11)
    
    df.drop(df.index[:1], inplace = True)
    
    return df

In [29]:
final_recommendation = get_recommendations(recommendation)

In [30]:
final_recommendation

,title,description,averageRating,origin,type
4918,The Hateful Eight,"Years after the Civil War, a bounty hunter and...",7.799989,Netflix,Movie
3656,The Hateful Eight: Extended Version,Trapped at a stagecoach stop as a storm rages ...,NaN,Netflix,TV Show
13154,Meeting Evil,John (Luke Wilson) is taken on a murder-fueled...,5.300000,Prime Video,Movie
18066,Mother And Child,"From writer-director Rodrigo Garcia (TV'S ""Big...",NaN,Prime Video,Movie
8457,Miss Peregrine's Home for Peculiar Children,A wildly imaginative fantasy-adventure about a...,6.700013,Disney +,Movie
7861,The Other Guys,New York cops Allen and Terry toil in obscurit...,6.705889,Netflix,Movie
3707,Unicorn Store,After failing out of art school and taking a h...,5.500000,Netflix,Movie
949,Dad Stop Embarrassing Me - The Afterparty,"Jamie Foxx, David Alan Grier and more from the...",NaN,Netflix,Movie
1392,Death to 2020,"As the year we all want to end finally does, t...",6.800000,Netflix,Movie
5891,Barely Lethal,A girl raised to be a government assassin just...,5.400000,Netflix,Movie


# Content Based Algorithm : Features and New

### New filtering

A remettre en forme. 

How long does it take for a film to become old? (value has to be put in days)

In [31]:
days_new = int(input("How long does it take for a film to become old ? : "))

date_new = dt.datetime.now()-dt.timedelta(days = days_new)

How long does it take for a film to become old ? : 300


### Filtering 

In [32]:
def loader_new(excel) :
    
    df = pd.DataFrame(pd.read_excel(excel))
    
    df = df.drop(["Unnamed: 0"], axis = 1)
    
    return df

In [33]:
def new_set_new(xlsx, df1) :
    
    df = loader_new(xlsx)
    
    df = df.reset_index()
    
    #Merging similarities
    
    df = df.merge(df1, how = 'left', left_on = 'index', right_on = 'index')
    
    return df

recommendation_new = new_set_new("streaming_content.xlsx", similarity)

In [34]:
def new_filtering (df, date) :
    
    df = df[df["date_added"] >= date]
    
    return df

recommendation_new = new_filtering(recommendation_new, date_new)

### Recommendation

In [35]:
def get_recommendations_new(df1) :
    
    df = df1.sort_values(by = ["similarity"], ascending = False)
    
    df = df[["title","description", "averageRating","origin","type"]].head(6)
    
    df.drop(df.index[:1], inplace = True)
    
    return df

final_new_recommendation = get_recommendations_new(recommendation_new)

In [36]:
final_new_recommendation

,title,description,averageRating,origin,type
307,Freedomland,When a woman reports that her infant son was k...,5.112932,Netflix,Movie
304,Deep Blue Sea,Scientists conduct research on sharks in searc...,5.904535,Netflix,Movie
19152,Peeples,Sparks fly when Wade Walker (Craig Robinson) c...,5.300000,Hulu,Movie
8318,Fantastic Four,Four astronauts develop superpowers and must j...,5.210624,Disney +,Movie
8408,Fantastic Four: Rise of the Silver Surfer,"The Fantastic Four meet The Silver Surfer, who...",5.642121,Disney +,Movie
